# Surface adsorption study using the ASE database
https://wiki.fysik.dtu.dk/ase/tutorials/db/db.html
In this tutorial we will adsorb C, N and O on 7 different FCC(111) surfaces with 1, 2 and 3 layers and we will use database files to store the results.

# Bulk 
First, we calculate the equilibrium bulk FCC lattice constants for the seven elements where the `EMT` potential works well:

In [1]:
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.eos import calculate_eos
from ase.db import connect

In [ ]:
db = connect('bulk.db')
for symb in ['Al', 'Ni', 'Cu', 'Pd', 'Ag', 'Pt', 'Au']:
    atoms = bulk(symb, 'fcc')
    atoms.calc = EMT()
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()  # find minimum
    # Do one more calculation at the minimu and write to database:
    atoms.cell *= (v / atoms.get_volume())**(1 / 3)
    atoms.get_potential_energy()
    db.write(atoms, bm=B)

In [ ]:
! ase db bulk.db -c +bm  # show also the bulk-modulus column

The `bulk.db` is an SQLite3 database in a single file.

In [ ]:
! file bulk.db

If you want to see what’s inside you can convert the database file to a json file and open that in your text editor:

In [ ]:
! ase db bulk.db --insert-into bulk.json

or, you can look at a single row like this:

In [ ]:
! ase db bulk.db Cu -j

The json file format is human readable, but much less efficient to work with compared to a SQLite3 file.

# Adsorbates
Now we do the adsorption calculations.

In [ ]:
from ase.calculators.emt import EMT
from ase.db import connect
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS

In [ ]:
db1 = connect('bulk.db')
db2 = connect('ads.db')

In [ ]:
def run(symb, a, n, ads):
    atoms = fcc111(symb, (1, 1, n), a=a)
    add_adsorbate(atoms, ads, height=1.0, position='fcc')

    # Constrain all atoms except the adsorbate:
    fixed = list(range(len(atoms) - 1))
    atoms.constraints = [FixAtoms(indices=fixed)]

    atoms.calc = EMT()
    opt = BFGS(atoms, logfile=None)
    opt.run(fmax=0.01)
    return atoms

In [ ]:
for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        for ads in 'CNO':
            atoms = run(symb, a, n, ads)
            db2.write(atoms, layers=n, surf=symb, ads=ads)

We now have a new database file with 63 rows.

In [ ]:
! ase db ads.db -n

These 63 calculations only take a few seconds with EMT. Suppose you want to use DFT and send the calculations to a supercomputer. In that case you may want to run several calculations in different jobs on the computer. In addition, some of the jobs could time out and not finish. It’s a good idea to modify the script a bit for this scenario. We add a couple of lines to the inner loop:

In [ ]:
for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        for ads in 'CNO':
            id = db2.reserve(layers=n, surf=symb, ads=ads)
            if id is not None:
                atoms = run(symb, a, n, ads)
                db2.write(atoms, layers=n, surf=symb, ads=ads)
                del db2[id]

The `reserve()` method will check if there is a row with the keys `layers=n`, `surf=symb` and `ads=ads`. If there is, then the calculation will be skipped. If there is not, then an empty row with those keys-values will be written and the calculation will start. When done, the real row will be written and the empty one will be removed. This modified script can run in several jobs all running in parallel and no calculation will be done twice.

In [ ]:
! ase db ads.db natoms=0 -c ++

Delete them, fix the problem and run the script again:

In [ ]:
! ase db ads.db natoms=0 --delete

In [ ]:
! ase db ads.db natoms=0

# Reference energies
Let’s also calculate the energy of the clean surfaces and the isolated adsorbates:

In [ ]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.db import connect
from ase.build import fcc111

In [ ]:
db1 = connect('bulk.db')
db2 = connect('ads.db')

In [ ]:
def run(symb, a, n):
    atoms = fcc111(symb, (1, 1, n), a=a)
    atoms.calc = EMT()
    atoms.get_forces()
    return atoms

Clean slabs:

In [ ]:
for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        id = db2.reserve(layers=n, surf=symb, ads='clean')
        if id is not None:
            atoms = run(symb, a, n)
            db2.write(atoms, id=id, layers=n, surf=symb, ads='clean')

Atoms:

In [ ]:
for ads in 'CNO':
    a = Atoms(ads)
    a.calc = EMT()
    a.get_potential_energy()
    db2.write(a)

In [ ]:
! ase db ads.db -n

Say we want those 24 reference energies (clean surfaces and isolated adsorbates) in a `refs.db` file instead of the big `ads.db` file. We could change the `refs.py` script and run the calculations again, but we can also manipulate the files using the `ase db` tool. First, we move over the clean surfaces:

In [ ]:
! ase db ads.db ads=clean --insert-into refs.db

In [ ]:
! ase db ads.db ads=clean --delete --yes

and then the three atoms (`pbc=FFF`, no periodicity):

In [ ]:
! ase db ads.db pbc=FFF --insert-into refs.db

In [ ]:
! ase db ads.db pbc=FFF --delete --yes

In [ ]:
! ase db ads.db -n

In [ ]:
! ase db refs.db -n

# Analysis
Now we have what we need to calculate the adsorption energies and heights:

In [ ]:
from ase.db import connect

In [ ]:
refs = connect('refs.db')
db = connect('ads.db')

In [ ]:
for row in db.select():
    ea = (row.energy -
          refs.get(formula=row.ads).energy -
          refs.get(layers=row.layers, surf=row.surf).energy)
    h = row.positions[-1, 2] - row.positions[-2, 2]
    db.update(row.id, height=h, ea=ea)

In [ ]:
! ase db ads.db Pt,layers=3 -c formula,ea,height